In [ ]:
%pip install -r ../requirements.txt

## Load libraries

In [ ]:
import warnings  # noqa: E402

from matplotlib import pyplot as plt  # noqa: E402
from omegaconf import OmegaConf  # noqa: E402

warnings.filterwarnings("ignore")

import sys  # noqa: E402

sys.path.insert(0, "../")

%matplotlib inline

from modules import config, utils  # noqa: E402

## Visualize dataset

In [ ]:
def view_image(ds):
    ds = iter(ds)
    image = next(ds)  # extract 1 batch from the dataset
    image = [utils.post_processing(img) for img in image]
    fig = plt.figure(figsize=(22, 22))
    for i in range(20):
        ax = fig.add_subplot(4, 5, i + 1, xticks=[], yticks=[])
        ax.imshow(image[i])

In [ ]:
# Data config path
data_config_path = "../conf/config.yaml"
cfg = OmegaConf.load(data_config_path)
load_confg = OmegaConf.to_container(cfg)
conf = config.LoadConfig(**load_confg)

In [ ]:
# Prepare dataloaders
dataset_train_class = utils.TensorflowDataset(train=True, dtype=conf.train.dtype, config=conf.data)
loader_train = utils.DataLoader(dataset_train_class, distributed=False)

## View

In [ ]:
view_image(loader_train())

## Logging image view

In [ ]:
import jax  # noqa: E402
import numpy as np  # noqa: E402

from modules import training, utils  # noqa: E402

In [ ]:
rng = jax.random.PRNGKey(0)
log_img_every_n_epoch = 1

for batch in loader_train():
    if len(batch) > 8:
        batch = batch[:8]
    generate_callback = training.GenerateCallback(batch, rng, every_n_epochs=log_img_every_n_epoch)
    break

In [ ]:
reconst_imgs = batch.copy()
imgs = np.stack([batch, reconst_imgs], axis=1).reshape(-1, *batch.shape[1:])
imgs = np.stack([utils.post_processing(img, resize=64) for img in imgs], axis=0)
img_to_log = utils.make_img_grid(imgs, nrows=2)
plt.imshow(img_to_log)
plt.show()